In [20]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from openai import OpenAI
import pandas as pd
from pydantic import BaseModel
from typing import Union
import dotenv
import os
dotenv.load_dotenv()

client_openai = OpenAI()

In [13]:
folder_path = 'hugo_data_samples'
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

In [16]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4.1"),
    dfs,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

response = agent.invoke("How many rows are there")
response



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "row_counts = {}\nfor i in range(1, 9):\n    df = globals()[f'df{i}']\n    row_counts[f'df{i}'] = len(df)\nrow_counts"}`


KeyError: 'df1'
Invoking: `python_repl_ast` with `{'query': "# Let's try to list all variables to see the available dataframe names\nlocals().keys()"}`


dict_keys(['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'row_counts', 'i'])
Invoking: `python_repl_ast` with `{'query': "# Now let's count the rows in each dataframe\ndf_row_counts = {f'df{i}': len(globals()[f'df{i}']) for i in range(1, 9)}\ndf_row_counts"}`


KeyError: 'df1'
Invoking: `python_repl_ast` with `{'query': "# Use locals() instead of globals() for the dataframe lookup\ndf_row_counts = {f'df{i}': len(locals()[f'df{i}']) for i in range(1, 9)}\ndf_row_counts"}`


{'df1': 39, 'df2': 39, 'df3': 100, 'df4': 200, 'df5': 1, 'df6': 39, 'df7': 123, 'df8': 98}Here are the number of rows in each dataframe:

- df1: 39 ro

{'input': 'How many rows are there',
 'output': 'Here are the number of rows in each dataframe:\n\n- df1: 39 rows\n- df2: 39 rows\n- df3: 100 rows\n- df4: 200 rows\n- df5: 1 row\n- df6: 39 rows\n- df7: 123 rows\n- df8: 98 rows\n\nIf you need the total or more details, let me know!'}

In [18]:
print(response["output"])

Here are the number of rows in each dataframe:

- df1: 39 rows
- df2: 39 rows
- df3: 100 rows
- df4: 200 rows
- df5: 1 row
- df6: 39 rows
- df7: 123 rows
- df8: 98 rows

If you need the total or more details, let me know!


In [ ]:
SCHEMA_DESCRIPTION = """
We have a procurement database with the following tables:

**Table 1: dispatch_parameters**
Description: This table contains information about the minimum stock levels, reorder quantities, and reorder intervals for different parts.
Columns:
- part_id
- min_stock_level
- reorder_quantity
- reorder_interval_days

**Table 2: material_master**
Description: This table provides detailed information about each part, including its name, type, models it is used in, dimensions, weight, and any related parts.
Columns:
- part_id (note: this is a primary key with format P123)
- part_name
- part_type
- used_in_models
- weight
- blocked_parts
- successor_parts
- comment

**Table 3: material_orders**
Description: This table records purchase orders for parts, including order details, supplier information, and delivery status.
Columns:
- order_id
- part_id
- quantity_ordered
- order_date
- expected_delivery_date
- supplier_id
- status
- actual_delivered_at

**Table 4: sales_orders**
Description: This table contains information about sales orders, including the model, version, quantity, order type, and dates related to the order process.
Columns:
- sales_order_id
- model
- version
- quantity
- order_type
- requested_date
- created_at
- accepted_request_date

**Table 5: stock_levels**
Description: This table provides the number of parts stored in different warehouse locations.
Columns:
- part_id
- part_name
- location
- quantity_available

**Table 6: stock_movements**
Description: This table records transactions related to inventory movements, including inbound and outbound quantities.
Columns:
- date
- part_id
- type
- quantity

**Table 7: suppliers**
Description: This table contains information about suppliers, including pricing, lead times, minimum order quantities, and reliability ratings for different parts.
Columns:
- supplier_id
- part_id
- price_per_unit
- lead_time_days
- min_order_qty
- reliability_rating

**Table 7: specs**
Description: This table contains information about the required parts to assemble a product, including the part name and the quantity required.
Columns:
- product_id
- product_name
- part_id
- quantity
"""

class Column(BaseModel):
    column_name: str
    value: Union[str, int, float, None]

class Row(BaseModel):
    table_name: str
    columns: list[Column]
    reasoning: str

class Response(BaseModel):
    rows: list[Row]
    
def generate_data_rows(input):
    prompt = f"""
    {SCHEMA_DESCRIPTION}

    Based on the schema description above and the input data below:
    {input}

    Your task is to:
    - Decide which rows need to be edited or inserted into the database.
    - For each row, clearly specify:
      - `table_name`: the table to insert or update
      - `columns`: a list of columns to modify, where each column has:
        - `column_name`: the exact field name from the schema
        - `value`: the new value to set for that column (can be a string, number, or null)
      - `reasoning`: a reason why the table is selected

    Important rules:
    - Only use tables and column names defined in the provided schema.
    - Fill missing information intelligently if the input clearly implies it.
    - Keep values consistent with their expected data types.
    - Return the final structured response strictly matching the given Response model format.

    Think carefully before constructing the rows.
    """

    completion = client_openai.beta.chat.completions.parse(
        model="gpt-4.1",
        response_format=Response,
        messages=[
            {
                "role": "system", 
                "content": 
                """
                You are a highly accurate procurement assistant. Your role is to map freeform input 
                into structured schema edits according to strict schema definitions. 
                You must not invent extra tables or columns.
                You must return clean, structured row edits exactly matching the database schema.
                """
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    rows = completion.choices[0].message.parsed.rows
    return rows

generate_data_rows("Add S4 V1 500W Brushless Motor parts to the material list")

[Row(table_name='material_master', columns=[Column(column_name='part_id', value='S4_V1_500W_Brushless_Motor'), Column(column_name='part_name', value='S4 V1 500W Brushless Motor'), Column(column_name='part_type', value='Brushless Motor'), Column(column_name='used_in_models', value='S4 V1'), Column(column_name='weight', value=None), Column(column_name='blocked_parts', value=None), Column(column_name='successor_parts', value=None), Column(column_name='comment', value=None)], reasoning='The task is to add the S4 V1 500W Brushless Motor as a new part. The material_master table stores part-level details such as id, name, type, applicable models, etc. Other columns are left null since the input provided no further details.')]